**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

### Sequence Diagram

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Account creation
    Platform->>Blockchain: Create Account
    Blockchain-->>Platform: Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
    
    
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, Blockchain: Set user details
    Platform->>Blockchain: Set User Details in Blockchain
    Blockchain-->>Platform: User Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *



In [2]:
DOMAIN = "test"

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

In [3]:
from integration_helpers import get_engine_receipts_address
# - Deploy contract
hash = create_account_contract()


2025-01-21 15:35:43.571 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_account_contract"
2025-01-21 15:35:43.577 | INFO     | iroha_helper:create_account_contract:41 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-21 15:35:43.578 | INFO     | iroha_helper:create_account_contract:41 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-21 15:35:44.503 | INFO     | iroha_helper:create_account_contract:41 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-21 15:35:44.534 | INFO     | iroha_helper:create_account_contract:41 - ('COMMITTED', 5, 0)
2025-01-21 15:35:44.536 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-21 15:35:44.556 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'293229de7113c19873bef1bcc287f123c30f3de32935a2a2ea5c19162fe9c3e6'
2025-01-21 15:35:44.557 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 7980A59EC1964BA75D50

2 - Creates a user account into the blockchain.

In [ ]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
address = integration_helpers.get_engine_receipts_address(hash)
create_user_account(address, user_account_short_id, DOMAIN, user_public_key, user_account)

# - Get account info
hash = get_account(address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(hash)


2025-01-21 15:35:44.611 | INFO     | __main__:<module>:13 - User Role: reviewer
2025-01-21 15:35:44.613 | INFO     | __main__:<module>:24 - Creating account with name: crazy_austin
2025-01-21 15:35:44.614 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-21 15:35:44.632 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'293229de7113c19873bef1bcc287f123c30f3de32935a2a2ea5c19162fe9c3e6'
2025-01-21 15:35:44.633 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 7980A59EC1964BA75D50F78E1FBD94803FE5D6D3
2025-01-21 15:35:44.634 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-21 15:35:44.635 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-01-21 15:35:44.658 | INFO     | iroha_helper:create_user_account:83 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-21 15:35:44.660 | INFO 

3 - Queries the blockchain to confirm the succesful creation of the account.

In [ ]:
#Query - GetAccountDetail



details = get_account_detail(f"{user_account_short_id}@{DOMAIN}")


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting.

In [ ]:
# - Deploys the contract
hash = create_detail_contract()

5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [ ]:
address = integration_helpers.get_engine_receipts_address(hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

6 - Queries the blockchain again to confirm the succesfull details setting.

In [ ]:
#Query - GetAccountDetail

details = get_account_detail(f"{user_account_short_id}@{DOMAIN}")


7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [ ]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
user_metadata_cid = details_dict["admin@test"]["user_json_ld_cid"]
logger.info(f"User Metadata CID: {user_metadata_cid}")

user_metadata = download_json_from_ipfs(user_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")